## JSON -> DataFrame

To convert the `JSON` structure to a `DataFrame`, we have to flatten it. Here we decided that
rows will be events. Thus columns should contain all metadata about the examinee that
generated the event (in database `SQL`, this is known as a "denormalized view" of the data).
On the up side, everything is based on events, which is simple. On the down side, we are
duplicating the examinee's info across all of her events.

Thus the transformed data will look like this:

```
Row 0: test session 0, event 0
Row 1: test session 0, event 1
...
Row n-1: test session 0, event n-1
Row n: test session 1, event 0
...

Each row represents: event of some examinee
Columns: {all examinee_attributes} followed by {all event_attributes}
```

Notes:
* In this data we happen to have only a single test session.
* Looks like there are two types of events: "interactions" and "events". Each one will have its own row.

What's hard to explain in words is easy when you print the resulting `DataFrame` and look at some of its rows.

In [49]:
import json 
import pandas as pd

In [ ]:
# Load the JSON file into a dictionary.

# Remember to use the 'with' clause around file access code. Keep the code inside the 'with'
# as small as possible, as it is good practice to relinquish resource holds as soon as you
# don't need them any more (they are fragile and may be needed/overwritten by other programs).
with open("homework_week2_json_parsing.json") as f:
    data = json.load(f)

In [ ]:
# Remember to modularize your code into small functions whenever possible. Don't put
# everything in one big loop that's hard to understand, debug and maintain!

# Global variables available to get_flattened_events() since they are defined outside
# its scope.
interaction_columns = ["interactionId", "interactionType", "value", "time"]
event_columns = ["type", "action", "time", "itemId", "from", "to"]
all_event_columns = ["event_type"] + interaction_columns + event_columns

In [ ]:
def get_flattened_events(heartbeat):
    """Returns a list of rows (not in a DataFrame, just a list) of interactions and events
    of an examinee's heartbeat field."""
    
    # Flatten the heartbeat fields into a list.
    heartbeats = [(k, v) for k, v in heartbeat.items() if k != "events" and k != "interactions"]

    # Flatten the interactions into a list.
    interactions = [(interaction_id, key, interaction[key]) 
         for interaction_id, interaction in enumerate(heartbeat["interactions"]) 
         for key in interaction_columns]

    # Flatten events.
    events = [(event["type"], event["action"], event["time"], event["itemId"],
               event["value"]["from"] if "value" in event else None,
               event["value"]["to"] if "value" in event else None)
               for event in heartbeat["events"]]

    # Prepend an event type column that's either "interaction" or "event". This is how I
    # interpreted putting the data into a DataFrame, but this can be done in other ways too.
    # This is basically a left-join: fill in all event columns of interaction rows with nulls,
    # and all interaction columns of event rows with nulls.
    all_event_data = [("interaction", ) + interaction +
                      tuple(len(event_columns) * [None]) 
                      for interaction in interactions] + \
    [("event", ) + tuple(len(interaction_columns) * [None]) + event for event in events]
    return all_event_data

In [ ]:
# Let's test this part of the code first. Find all events of session 0.
all_events_example = get_flattened_events(data['TestSessions'][0]['HeartBeats'][0])

# Some interaction events.
print(all_events_example[:3])

# Now show some "event-events" from the above table.
print(all_events_example[-3:])

In [ ]:
# Gather the metadata of each test session and examinee.
# Remember - vectorized expressions are more readable and often faster in Python than loops, 
# to use them when you can.

# Instead of hard-coding those, just read column names from the first examinee's dictionary
# except the heartbeat field.
examinee_columns = [field for field in data['TestSessions'][0].keys() 
                    if field != "HeartBeats"]

def get_examinee_info(session):
    """Returns examinee metadata for a certain session."""
    # The values corresponding to 'examinee_columns'. Note that items() returns them in a
    # deterministic order, so we can rely on their matching.
    return tuple(value 
                 for field, value in data['TestSessions'][0].items() 
                 if field != "HeartBeats")

In [ ]:
# Let's test only this part of the code.
print(get_examinee_info(data['TestSessions'][0]))

In [ ]:
# Now calculate the cross-product of examinees and events.

def get_examinee_event_data_frame(data):
    """Returns a DataFrame whose rows are events, for each examinee and interaction/event
    entry in the JSON data.
    
    Adds a column with the session number (a counter starting at 0 for all sessions in the
    JSON data).
    
    Assumes a single HeartBeats field per session."""

    all_columns = ["session_id"] + examinee_columns + all_event_columns
    
    # Note: this runs over the entire data only once, which is ideally what we want to do,
    # especially if the data is being streamed.
    df = pd.DataFrame(
        [(session_id,) + get_examinee_info(session) + event
         for session_id, session in enumerate(data['TestSessions'])
         for event in get_flattened_events(session['HeartBeats'][0])],
        columns = all_columns)
    
    return df

In [ ]:
df = get_examinee_event_data_frame(data)

In [ ]:
df

In [ ]:
df.info()

In [ ]:
# An example query. Doesn't make a lot of sense here but just to see what's possible.
df.groupby('ExamineeIdentifier').mean()

# XML -> DataFrame
This follows the same idea. We do a simple example of extracting test event timestamps for each customer.

We use an interesting idea here: traversing the entire XML tree with a recursive call. When we arrive at a node of interest, we emit it (with the "yield" keywords; we learned about generator expressions - this is how you write one. Every time "yield" is called, an element is emitted when the function is called).

Then, we go back up the hierarchy for each such node of interest (say, an event node), and extract the customer information corresponding to it from elsewhere in the hierarchy. W

In [52]:
import xml.etree.ElementTree as ET
tree = ET.parse('homework_week2_xml_parsing.xml')
root = tree.getroot()

In [55]:
"""Helper functions."""
def unqualified_tag_name(tag):
    return tag.split("}")[1]

def first_child_with_name(node, child_name):
    return next(child for child in node
                if unqualified_tag_name(child.tag) == child_name)

def get_nested_element_data(element):
    return [(unqualified_tag_name(c.tag), c.text.strip() if c.text else "") 
            for c in element if c.text and len(c.text.strip()) > 0]

"""Traverses the entire XML tree and generates elements that have actual
fields in them (i.e., leaves of the tree). For each node, we output
1) The breadcrumb hierarchy (all its ancestors, starting with the root
node); 2) its nested field names and values.

Refer to the python manual to learn more about the yield keyword, which
is how to emit elements from a function that returns a generator expression,
like the one below."""


def traverse_xml_tree(root):
    """Top-level call."""
    for value in _traverse_xml_tree(root, []):
        yield value

def _traverse_xml_tree(element, stack):
    """Recursive call. We pass the stack when traversing children, tacking
    on the current node."""
    stack.append(element)
    fields = get_nested_element_data(element)
    if fields:
        # Stack must be deep-copied, since it is a changing list throughout
        # the recursion.
        yield stack.copy(), fields
    for child in element:
        for value in _traverse_xml_tree(child, stack):
            yield value
    stack.pop()
    
def get_customer_name(record):
    """Walks down the hierarchy from the root of the current record
    and returns the customer first and last name that appear in nested
    tags under the root."""
    event_data = first_child_with_name(record[0][0], "CustomerResultData")
    customer_name = first_child_with_name(event_data, "CustomerName")
    return [customer_name[0].text, customer_name[1].text]

def extract_time_session_events(root):
    """The main parsing call. Joins the customer and test data."""
    records = [entry for entry in traverse_xml_tree(root) if 
         [unqualified_tag_name(parent.tag) for parent in entry[0]] ==
         ['CustomerRecord', 'CustomerEventData', 'TestSessionEvent']]

    columns = ['FirstName', 'LastName', 'SequenceNo', 'TimeStamp', 'TestComponentCode', 'DetailText']
    data = [get_customer_name(record) +  [value for key, value in record[1]] for record in records]
    return pd.DataFrame(data, columns = columns)

In [56]:
extract_time_session_events(root)

,FirstName,LastName,SequenceNo,TimeStamp,TestComponentCode,DetailText
0,85c24d74e0f90af,d79ee6182f3bd06,1,2019-04-13T10:01:19.716+02:00,TELXMLUNF00011000PKG001,Candidate starts the test
1,85c24d74e0f90af,d79ee6182f3bd06,884,2019-04-13T13:49:11.097+02:00,TELXMLUNF00011000PKG001,Candidate completes the test


Of course, one could parse different parts of the file (e.g., TestEvents). This would be done similarly.